In [1]:
#enveloping potential
# simple Example plot Enveloped Potential with two Harmonic Oscilators
##Imports:
import os, sys as csys, datetime, tempfile
import math, numpy as np, pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm


csys.path.append(os.getcwd()+"/../../..")

#Ensembler
import ensembler.potentials.TwoD as pot2D
import ensembler.potentials.TwoD as pot2D

from ensembler.samplers.stochastic import metropolisMonteCarloIntegrator
from ensembler.conditions.box_conditions import periodicBoundaryCondition
from ensembler.system.basic_system import system

import ensembler.visualisation.plotPotentials as exPlot

plot_resolution= 1200 # lowRes but fast: 120
%matplotlib inline

In [2]:
#ENERGIES Sampling - CURRENTLY NOT USED!
def write_out_etraj(traj, out_path, V1, V2):
    visited_positions =  np.array(list(map(np.array, traj.position)))
    state1 = V1.ene(visited_positions)
    state2 = V2.ene(visited_positions)
    Vrenergies = np.array(traj.total_potential_energy)

    out_file = open(out_path, "w")
    out_file.write("t\tV1\tV2\tVr\n")#HEADER
    for t, V1, V2, Vr in zip(traj.index, state1, state2, Vrenergies):
        out_file.write(str(t)+"\t"+str(V1)+"\t"+str(V2)+"\t"+str(Vr)+"\n")

    out_file.close()

In [8]:
sys = None
def simulate_replicas(out_dir:str, barrier, opt_s, replicas=10, simulation_steps=1000000):
    if(not os.path.exists(out_dir)):
        os.mkdir(out_dir)
    
    start_t = datetime.datetime.now()
    #Potentials
    amplitude = barrier/2
    shift  = np.rad2deg(0.5*math.pi)
    Eoff = [0,0]

    V1 = pot2D.wavePotential(phase_shift=(shift,shift), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude))
    V2 = pot2D.wavePotential(phase_shift=(0.0, 0.0), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude))
    edsPot = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)
    exPlot.plot_2D_2State_EDS_potential(eds_pot=edsPot, out_path = out_dir+"/SimSpace_"+str(barrier)+"kT.png", point_resolution=plot_resolution)

    #Simulation Setup
    svals = sorted([10, 1, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01]+[opt_s], reverse=True)
    ##Build modules
    periodic_bound = periodicBoundaryCondition(boundary=[[-180,180], [-180,180]])
    integrator = metropolisMonteCarloIntegrator(fixed_step_size=[1,1], max_iteration_tillAccept=1000) 
    
    ###Build System
    sys=system(potential=edsPot, sampler=integrator, conditions=[periodic_bound])
    print(sys.nDimensions, edsPot.constants[edsPot.nDimensions])
    print("svalue sequence - #s="+str(len(svals))+" ", svals)
    
    ## SIMULATE
    start=0
    syst = None
    for replica in tqdm(range(start, replicas), desc="Replicas: ", leave=False):
        ##INIT REPLICA
        ###fILE hANDLING
        replica_out = out_dir+"/replica_"+str(replica)
        if(not os.path.exists(replica_out)):
            os.mkdir(replica_out)

        ## SIMULATE REPLICA
        start_sim = datetime.datetime.now()
        s_val_posDict = {}  #OUT
        for s in tqdm(svals, desc="Current Simulation of replica "+str(replica)+": ", leave=False):
            sys.potential.s = s
            cur_state = sys.simulate(simulation_steps, withdraw_traj=True, init_system=True)

            s_val_posDict.update({s:sys.trajectory})

        end_sim = datetime.datetime.now()
        duration_sim = end_sim-start_sim

        traj = sys.trajectory

        #plotting:
        print("plotting")
        start_plot = datetime.datetime.now()
        exPlot.plot_2D_2State_EDS_potential(edsPot, out_path = replica_out+"/SimSpace_withTraj_"+str(barrier)+"kT.png", traj=traj, point_resolution=plot_resolution)
        if(replica == 0):
            fig = exPlot.plot_2D_2State_EDS_potential_sDependency(sVal_traj_Dict=s_val_posDict, eds_pot=edsPot, plot_trajs=False, out_path=out_dir+"/s_dependent_sampling_relBarrier_"+str(barrier)+"kT.png", point_resolution=plot_resolution)
        fig = exPlot.plot_2D_2State_EDS_potential_sDependency(sVal_traj_Dict=s_val_posDict, eds_pot=edsPot, plot_trajs=True, out_path=replica_out+"/s_dependent_sampling_relBarrier_"+str(barrier)+"kT_withTraj.png", point_resolution=plot_resolution)
        end_plot = datetime.datetime.now()
        duration_plot = end_plot-start_plot

        for s in s_val_posDict:
            write_out_etraj(traj=s_val_posDict[s], out_path=replica_out+"/replica_traj_s"+str(s)+".dat", V1=sys.potential.V_is[0], V2=sys.potential.V_is[1])
        sys.write_trajectory(replica_out+"/total_replica_traj.dat")
        break

    del edsPot, sys, traj 
    
    end_t = datetime.datetime.now()
    duration = end_t-start_t
    print("Done - duration: ", duration)
    print("Done - simulation duration: ", duration_sim)
    print("Done - plotting duration: ", duration_plot)



In [9]:
#run multiple replicas
tmp_dir = tempfile.gettempdir()+"/edsSim"
if(not os.path.exists(tmp_dir)):
    os.mkdir(tmp_dir)
os.chdir(tmp_dir)

print(tmp_dir)


#BUILD Potential:
#params:
replicas = 1#0

##STEPS
each_sim = 100#00000 #each s value and each replica

#s_est = 0.609/Delta U_barrier
optimal_ses = {2.5: 0.122,
               5: 0.061,
               10: 0.03,
               20: 0.03, #0.015
               50: 0.006}


for barrier in tqdm(optimal_ses, desc="Barrier approaches: "):
    approach_dir = tmp_dir+"/independent_simulations_with_"+str(barrier)+"kT_barriers"
    simulate_replicas(out_dir=approach_dir, barrier=barrier, opt_s=optimal_ses[barrier], replicas=replicas, simulation_steps=each_sim)



C:\Users\benja\AppData\Local\Temp/edsSim


-3.141592653589793 3.141592653589793 -3.141592653589793 3.141592653589793
2 2
svalue sequence - #s=9  [10, 1, 0.5, 0.25, 0.122, 0.1, 0.05, 0.025, 0.01]


Simulation:  Simulation: 100%|██████████| 100/100 [00:01<00:00, 55.71it/s]


plotting
-3.141592653589793 3.141592653589793 -3.141592653589793 3.141592653589793
single [ -8.69763621  -7.58871512  -8.58871512  -9.58871512  -8.58871512
  -9.58871512  -8.58871512  -7.58871512  -8.58871512  -7.58871512
  -8.58871512  -9.58871512 -10.58871512 -11.58871512 -10.58871512
  -9.58871512 -10.58871512 -11.58871512 -12.58871512 -13.58871512
 -14.58871512 -13.58871512 -14.58871512 -13.58871512 -12.58871512
 -11.58871512 -10.58871512 -11.58871512 -10.58871512 -11.58871512
 -10.58871512  -9.58871512  -8.58871512  -7.58871512  -8.58871512
  -7.58871512  -8.58871512  -9.58871512  -8.58871512  -7.58871512
  -6.58871512  -5.58871512  -4.58871512  -3.58871512  -4.58871512
  -3.58871512  -4.58871512  -3.58871512  -4.58871512  -5.58871512
  -4.58871512  -3.58871512  -4.58871512  -5.58871512  -4.58871512
  -5.58871512  -4.58871512  -5.58871512  -4.58871512  -5.58871512
  -6.58871512  -5.58871512  -4.58871512  -3.58871512  -2.58871512
  -1.58871512  -2.58871512  -1.58871512  -0.58871512

[-4.13931472  4.35302311  5.35302311  6.35302311  5.35302311  6.35302311
  7.35302311  8.35302311  7.35302311  8.35302311  7.35302311  8.35302311
  7.35302311  8.35302311  7.35302311  8.35302311  7.35302311  8.35302311
  7.35302311  8.35302311  7.35302311  6.35302311  7.35302311  6.35302311
  7.35302311  8.35302311  7.35302311  8.35302311  9.35302311  8.35302311
  7.35302311  6.35302311  5.35302311  4.35302311  5.35302311  4.35302311
  5.35302311  4.35302311  5.35302311  4.35302311  3.35302311  4.35302311
  3.35302311  4.35302311  5.35302311  4.35302311  3.35302311  4.35302311
  5.35302311  4.35302311  3.35302311  2.35302311  1.35302311  0.35302311
 -0.64697689  0.35302311  1.35302311  2.35302311  1.35302311  0.35302311
  1.35302311  2.35302311  1.35302311  0.35302311 -0.64697689 -1.64697689
 -0.64697689 -1.64697689 -2.64697689 -1.64697689 -0.64697689 -1.64697689
 -0.64697689 -1.64697689 -0.64697689  0.35302311 -0.64697689 -1.64697689
 -0.64697689 -1.64697689 -0.64697689 -1.64697689 -0

...

[-4.90500273  8.97075812  7.97075812  6.97075812  7.97075812  6.97075812
  7.97075812  6.97075812  7.97075812  6.97075812  7.97075812  6.97075812
  7.97075812  6.97075812  7.97075812  6.97075812  7.97075812  8.97075812
  7.97075812  8.97075812  7.97075812  6.97075812  7.97075812  6.97075812
  7.97075812  8.97075812  7.97075812  6.97075812  7.97075812  8.97075812
  7.97075812  6.97075812  7.97075812  8.97075812  7.97075812  6.97075812
  7.97075812  6.97075812  7.97075812  6.97075812  7.97075812  6.97075812
  7.97075812  6.97075812  7.97075812  6.97075812  7.97075812  6.97075812
  7.97075812  8.97075812  7.97075812  8.97075812  7.97075812  6.97075812
  7.97075812  8.97075812  7.97075812  6.97075812  7.97075812  8.97075812
  9.97075812  8.97075812  7.97075812  8.97075812  7.97075812  6.97075812
  7.97075812  6.97075812  7.97075812  8.97075812  7.97075812  8.97075812
  7.97075812  8.97075812  7.97075812  6.97075812  7.97075812  6.97075812
  7.97075812  6.97075812  7.97075812  6.97075812  7

Simulation:  Simulation: 100%|██████████| 100/100 [00:02<00:00, 40.62it/s]


plotting
-3.141592653589793 3.141592653589793 -3.141592653589793 3.141592653589793
single [ -0.71734671 -10.67342441 -11.67342441 -10.67342441 -11.67342441
 -10.67342441 -11.67342441 -10.67342441 -11.67342441 -10.67342441
 -11.67342441 -10.67342441 -11.67342441 -10.67342441 -11.67342441
 -10.67342441 -11.67342441 -10.67342441 -11.67342441 -10.67342441
 -11.67342441 -10.67342441 -11.67342441 -10.67342441 -11.67342441
 -10.67342441 -11.67342441 -10.67342441 -11.67342441 -10.67342441
 -11.67342441 -10.67342441 -11.67342441 -10.67342441 -11.67342441
 -10.67342441 -11.67342441 -10.67342441 -11.67342441 -10.67342441
 -11.67342441 -10.67342441 -11.67342441 -10.67342441 -11.67342441
 -10.67342441 -11.67342441 -10.67342441 -11.67342441 -10.67342441
 -11.67342441 -10.67342441 -11.67342441 -10.67342441 -11.67342441
 -10.67342441 -11.67342441 -10.67342441 -11.67342441 -10.67342441
 -11.67342441 -10.67342441 -11.67342441 -10.67342441 -11.67342441
 -10.67342441 -11.67342441 -10.67342441 -11.67342441

[-2.2869811  -1.53026526 -0.53026526  0.46973474  1.46973474  0.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  0.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  0.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1.46973474  2.46973474
  1.46973474  2.46973474  1.46973474  2.46973474  1

[ 0.93231903 -1.71734671 -2.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0.71734671 -1.71734671
 -0.71734671 -1.71734671 -0.71734671 -1.71734671 -0

In [10]:
approach_dir

'C:\\Users\\benja\\AppData\\Local\\Temp/edsSim/independent_simulations_with_50kT_barriers'

In [11]:
sys
